In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ipdb
torch.manual_seed(1)
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
# from IPython.core.debugger import Pdb
# debug_here = Tracer()
# inputs.shape

In [11]:
lstm.num_layers

1

In [12]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
#     ipdb.set_trace()

In [13]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(inputs.shape)
# print(hidden.shape)
print(out.shape)
print(hidden)

torch.Size([5, 1, 3])
torch.Size([5, 1, 3])
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


In [14]:
rnn = nn.LSTM(10, 20,2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [15]:
rnn = nn.LSTM(125, 80,2)
input = torch.randn(8, 1, 125)# 一个句子，8个单词 每个单词维度为125
h0 = torch.randn(2, 1, 80)
c0 = torch.randn(2, 1, 80)
output, (hn, cn) = rnn(input, (h0, c0))

In [16]:
rnn

LSTM(125, 80, num_layers=2)

In [17]:
input.shape


torch.Size([8, 1, 125])

In [18]:
output.shape

torch.Size([8, 1, 80])

In [19]:
hn.shape

torch.Size([2, 1, 80])

In [20]:
cn.shape

torch.Size([2, 1, 80])

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch as t
from torch import nn
import numpy as np
import torch.optim as optim

try:
    import ipdb
except:
    import pdb as ipdb

In [24]:
    class Conv2dBlock(nn.Module):
        def __init__(self,ninp,fmaps,kwidth,stride):
            super().__init__()
            self.conv=nn.Conv2d(ninp,fmaps,kwidth,stride)
            self.pool=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
            self.act=nn.ELU()

        def forward(self,x):
            x=self.conv(x)
            x=self.pool(x)
            x=self.act(x)
            return x

    class Conv1dBlock(nn.Module):
        def __init__(self,ninp,noutp,kernel_size,stride=None,padding=0,Handle=True):
            super(Conv1dBlock,self).__init__()
            self.conv=nn.Conv1d(ninp,noutp,kernel_size,stride=stride,padding=padding)
            self.Handle=Handle
            self.norm=nn.BatchNorm1d(noutp)
            self.pool=nn.MaxPool1d(5,stride=1,padding=2)
            self.act=nn.ReLU()
            self.dropout=nn.Dropout()
        def forward(self,x):
            x=self.conv(x)
            if self.Handle:
                x=self.norm(x)
                x=self.pool(x)
                x=self.act(x)
                x=self.dropout(x)
            return x



In [25]:

class PPGNet(nn.Module):
    """
    网络结构定义
    """
    '''
    def __init__(self,ntr,nc):
        super(PPGNet,self).__init__()
#       ngf=opt.ngf # 
        self.ntr=ntr #每一段信号追踪的分片数
        self.nc=nc #中间卷积层的个数
        self.ConvBlocks=nn.ModuleList()
        self.conv1=nn.Conv2d(4,8,1)
        self.conv2=nn.Conv2d(8,16,(ntr,3))
        self.act2=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
        for i in np.arange(nc):
            ConvBlock=Conv2dBlock(2**(i+4),2**(i+5),(1,3),(1,1))
            self.ConvBlocks.append(ConvBlock)
        self.conv3=nn.Conv2d(2**(nc+4),32,(1,1),(1,1))
        self.act3=nn.ELU()
        self.fc=nn.Sequential(
            nn.Linear(64,512),
            nn.ELU(),
            nn.Linear(512,1),
            nn.ELU()
        )
    '''
    def __init__(self,fmaps,input_dim1,input_dim2,hidden_dim):
            super(PPGNet,self).__init__()

            # fisrt CNN feature extractor
            self.hidden_dim=hidden_dim
            self.ParallelBlocks=nn.ModuleList()
            for idx,kernel in enumerate(fmaps,start=1):
                ConvBlock=Conv1dBlock(1,1,kernel,stride=2,Handle=False)
                self.ParallelBlocks.append(ConvBlock)
            # lstm feature extractor      
            self.lstm1=nn.LSTM(input_dim1,hidden_dim,2) #   
            self.lstm2=nn.LSTM(input_dim2,hidden_dim,2)# 
        
           # middle sequential blocks 
            self.ConvBlocks=nn.ModuleList()
            ConvBlock1=Conv1dBlock(1,32,40,stride=1)
            self.ConvBlocks.append(ConvBlock1)
#             for jdx in np.arange(8):
            ConvBlock2=Conv1dBlock(32,47,50,stride=1)
            self.ConvBlocks.append(ConvBlock2)
        
            # linear
            self.linear=nn.Sequential(
                nn.Linear(384*125,1),
                nn.ReLU()
            )
                
#             self.conv2=nn.Conv2d(8,16,(ntr,3))
#             self.act2=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
#             for i in np.arange(nc):
#                 ConvBlock=Conv2dBlock(2**(i+4),2**(i+5),(1,3),(1,1))
#                 self.ConvBlocks.append(ConvBlock)
#             self.conv3=nn.Conv2d(2**(nc+4),32,(1,1),(1,1))
#             self.act3=nn.ELU()
#             self.fc=nn.Sequential(
#                 nn.Linear(64,512),
#                 nn.ELU(),
#                 nn.Linear(512,1),
#                 nn.ELU()
#             )

    def forward(self,x):
#         channel=x.size(1)
#         ipdb.set_trace()
#         print('1:',x.shape)
#         length=len(self.ParallelBlocks)
        ParallelOut=[]
        channelout=[]#每一个通道平行卷积后处理
        serialout=[]
        for chidx in np.arange(x.size(1)):
            for i in np.arange(len(self.ParallelBlocks)):
                Block=self.ParallelBlocks[i]
                InitOut=Block(x[:,chidx,:].unsqueeze(1))
                ParallelOut.append(InitOut)

            channelout=torch.cat(ParallelOut,2)
            ParallelOut=[]
            # sequential block
#             ipdb.set_trace()

            for chidy in np.arange(len(self.ConvBlocks)):
                SecondBlock=self.ConvBlocks[chidy]
                if chidy == 0:
                    SecondOut=SecondBlock(channelout)
                else:
                    SecondOut=SecondBlock(SecondOut)

                if chidy+1==len(self.ConvBlocks):
                    serialout.append(SecondOut)
        # 5 channel concat
    #   SecondIn=torch.cat(ParallelOut,1) # n*40*__
        
        
                
        
#         X=t.cat(ParallelOut,1)
#         X=self.conv1(X)
            
#         ipdb.set_trace()
#         for j in np.arange(len(self.ConvBlocks)):
#             Block=self.ConvBlocks[j]
#             X=Block(X)
#         ipdb.set_trace()
        hidden=(torch.zeros([2, x.size(0), self.hidden_dim],dtype=torch.double),
                torch.zeros([2, x.size(0), self.hidden_dim],dtype=torch.double))
        LstmOut,self.hidden=self.lstm1(x.view(x.size(1),-1,x.size(2)),hidden)
        
        concat1=torch.cat(serialout,1)
        LstmOut1=LstmOut.view(LstmOut.size(1),-1,LstmOut.size(2))
        concat=torch.cat((LstmOut1,concat1),1)
        
        hidden=(torch.zeros([2, concat.size(0), self.hidden_dim],dtype=torch.double),
                torch.zeros([2, concat.size(0), self.hidden_dim],dtype=torch.double))
        LstmOutp,self.hidden=self.lstm1(concat.view(concat.size(1),-1,concat.size(2)),hidden)
#         ipdb.set_trace()
        out=self.linear(LstmOutp.view(LstmOutp.size(1),-1))
        return out
#         x1=self.conv1(x)
#         print('2:',x.shape)
#         x=self.conv2(x)
#         print('3:',x.shape)
#         x=self.act2(x)
#         print('4:',x.shape)
#         pdb.set_trace()
#         for i in np.arange(self.nc):
#             ConvBlock=self.ConvBlocks[i]
#             x=ConvBlock(x)
#             print('{}:'.format(i+5),x.shape)
#         pdb.set_trace()
#         x=self.conv3(x)
#         x=self.act3(x)
#         x=x.view(x.size(0),1,-1)
#         out=self.fc(x)
#         return out.squeeze()
    
    def  get_n_params(self):
        pp=0
        for p in list(self.parameters()):
            nn=1
            for s in list(p.size()):
                nn = nn*s
            pp += nn
        return pp  




In [26]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2dBlock') != -1:
        print('Initializing weights of convresblock to 0.0, 0.02')
        for k, p in m.named_parameters():
            if 'weight' in k and 'conv' in k:
                p.data.normal_(0.0, 0.02)
    elif classname.find('Conv2d') != -1:
        print('Initialzing weight to 0.0, 0.02 for module: ', m)
        m.weight.data.normal_(0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            print('bias to 0 for module: ', m)
            m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        print('Initializing FC weight to xavier uniform')
        nn.init.xavier_uniform_(m.weight.data)
def build_norm_layer(norm_type, param=None, num_feats=None):
    if norm_type == 'bnorm':
        return nn.BatchNorm2d(num_feats)
    elif norm_type == 'snorm':
        spectral_norm(param)
        return None
    elif norm_type is None:
        return None
    else:
        raise TypeError('Unrecognized norm type: ', norm_type)

In [27]:
kernels=[5,20,40,60,80]
model=PPGNet(kernels,125,125,125)

print('Total model parameters: ',model.get_n_params())
# print(model)

Total model parameters:  628938


In [28]:
    from torch.utils import data
    import os
    import scipy.io as sio
    class PPGData(data.Dataset):
        def __init__(self,root):
            datas=os.listdir(root)
            self.totaldata=[os.path.join(root,data) for data in datas]
        def __getitem__(self,index):
    #       timedata=np.zeros((4,7,1025))
#             sipdb.set_trace()
            ppgpath=self.totaldata[index]
            time=sio.loadmat(ppgpath)
            time=time['ppg']
            time=time.reshape(8,125)
            listslice=ppgpath.split('-')
            label=listslice[2][:-4]
            return time,label
        def __len__(self):
            return len(self.totaldata)



In [34]:

from torch.utils.data import DataLoader
filepath='/home/wcj/ReferenceProject/PPGnet/ppghrnormalization'
dataloader=PPGData(filepath)
# pdb.set_trace()
device = 'cpu'
if t.cuda.is_available:
#     device = t.device('cuda:0' if t.cuda.is_available() else 'cpu')
    device = 'cuda'
CUDA = (device == 'cuda')
print(device,CUDA)
# pdb.set_trace()
dloador=DataLoader(dataloader,batch_size=10,shuffle=True,num_workers=8,pin_memory=CUDA)

print(len(dataloader))
dataiter=iter(dloador)
sample,label=dataiter.next()

cuda True
1622


In [36]:
Net.apply(weights_init)
##################
#  lr=1e-3 afternormalization
##################
epochnum=0
iteration=0
dloador=DataLoader(dataloader,batch_size=100,shuffle=True,num_workers=8,pin_memory=CUDA)

optimizer=optim.Adam(model.parameters(),lr=1e-3)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(200):
    epochnum+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         ipdb.set_trace()
        iteration+=1
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#         if i%10==9:
        print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss))
        running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0501Net_epoch%d-iteration%d.pth'%(epochnum,iteration))

[1,    1] loss 134.092827
[1,    2] loss 129.274290
[1,    3] loss 128.791579
[1,    4] loss 118.235663
[1,    5] loss 105.868375
[1,    6] loss 74.809943
[1,    7] loss 43.395651
[1,    8] loss 19.855660
[1,    9] loss 45.413553
[1,   10] loss 53.809704
[1,   11] loss 45.876168
[1,   12] loss 23.412966
[1,   13] loss 17.809565
[1,   14] loss 27.122727
[1,   15] loss 29.639796
[1,   16] loss 32.432791
[1,   17] loss 32.789542
[2,    1] loss 23.644853
[2,    2] loss 22.780210
[2,    3] loss 21.503678
[2,    4] loss 19.429078
[2,    5] loss 25.493596
[2,    6] loss 23.965543
[2,    7] loss 25.221651
[2,    8] loss 18.932323
[2,    9] loss 20.856776
[2,   10] loss 19.119598
[2,   11] loss 18.913142
[2,   12] loss 21.213000
[2,   13] loss 20.959008
[2,   14] loss 19.532109
[2,   15] loss 19.702069
[2,   16] loss 19.569937
[2,   17] loss 21.859047
[3,    1] loss 20.085658
[3,    2] loss 15.880426
[3,    3] loss 23.686701
[3,    4] loss 19.109917
[3,    5] loss 18.507562
[3,    6] loss 18.41

KeyboardInterrupt: 

In [37]:
# weights_init(model)
##################
#  lr=1e-4 afternormalization
##################
# epochnum=0
# iteration=0
dloador=DataLoader(dataloader,batch_size=100,shuffle=True,num_workers=8,pin_memory=CUDA)

optimizer=optim.Adam(model.parameters(),lr=1e-4)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(200):
    epochnum+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         ipdb.set_trace()
        iteration+=1
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#         if i%10==9:
        print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss))
        running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0501Net_epoch%d-iteration%d.pth'%(epochnum,iteration))

[1,    1] loss 16.211209
[1,    2] loss 18.107575
[1,    3] loss 18.357925
[1,    4] loss 18.324435
[1,    5] loss 16.720637
[1,    6] loss 18.338521
[1,    7] loss 19.285154
[1,    8] loss 17.622786
[1,    9] loss 18.366259
[1,   10] loss 18.893629
[1,   11] loss 19.094107
[1,   12] loss 19.132169
[1,   13] loss 18.367290
[1,   14] loss 20.716530
[1,   15] loss 19.003687
[1,   16] loss 17.385453
[1,   17] loss 14.196033
[2,    1] loss 18.235874
[2,    2] loss 16.567494
[2,    3] loss 16.401013
[2,    4] loss 17.091832
[2,    5] loss 19.506608
[2,    6] loss 20.021504
[2,    7] loss 17.635600
[2,    8] loss 17.978462
[2,    9] loss 18.155143
[2,   10] loss 14.680262
[2,   11] loss 16.439854
[2,   12] loss 17.612507


Traceback (most recent call last):
  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [39]:
# weights_init(model)
# Net.apply(weights_init)

##################
#  lr=1e-5 afternormalization
##################
epochnum=0
iteration=0
dloador=DataLoader(dataloader,batch_size=100,shuffle=True,num_workers=8,pin_memory=CUDA)

optimizer=optim.Adam(model.parameters(),lr=1e-5)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(50):
    epochnum+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         ipdb.set_trace()
        iteration+=1
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#         if i%10==9:
        print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss))
        running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0501Net_epoch%d-iteration%d.pth'%(epochnum,iteration))

[1,    1] loss 17.891596
[1,    2] loss 17.157533
[1,    3] loss 17.042480
[1,    4] loss 16.967996
[1,    5] loss 15.609225
[1,    6] loss 18.509522
[1,    7] loss 15.846373
[1,    8] loss 14.574996
[1,    9] loss 18.512734
[1,   10] loss 17.848633
[1,   11] loss 16.317942
[1,   12] loss 17.116231
[1,   13] loss 15.721198
[1,   14] loss 16.113704
[1,   15] loss 14.936858
[1,   16] loss 15.386524
[1,   17] loss 15.900111
[2,    1] loss 16.811064
[2,    2] loss 15.158420
[2,    3] loss 17.331233
[2,    4] loss 17.252523
[2,    5] loss 17.641889
[2,    6] loss 14.640988
[2,    7] loss 15.988531
[2,    8] loss 18.585344
[2,    9] loss 15.400597
[2,   10] loss 17.280231
[2,   11] loss 15.688201
[2,   12] loss 15.065075
[2,   13] loss 14.124302
[2,   14] loss 17.115859
[2,   15] loss 17.738061
[2,   16] loss 17.447287
[2,   17] loss 17.219784
[3,    1] loss 14.572923
[3,    2] loss 18.592610
[3,    3] loss 15.993784
[3,    4] loss 16.348481
[3,    5] loss 16.737167
[3,    6] loss 18.471870


[19,   16] loss 14.364961
[19,   17] loss 13.147450
[20,    1] loss 15.801803
[20,    2] loss 14.173079
[20,    3] loss 13.975084
[20,    4] loss 17.965181
[20,    5] loss 12.425075
[20,    6] loss 14.626095
[20,    7] loss 17.265002
[20,    8] loss 14.614478
[20,    9] loss 16.299786
[20,   10] loss 14.949564
[20,   11] loss 15.338228
[20,   12] loss 13.548396
[20,   13] loss 14.594032
[20,   14] loss 15.990938
[20,   15] loss 14.911098
[20,   16] loss 16.287061
[20,   17] loss 13.060734
[21,    1] loss 15.170767
[21,    2] loss 14.147226
[21,    3] loss 14.180678
[21,    4] loss 17.187362
[21,    5] loss 14.445876
[21,    6] loss 12.994238
[21,    7] loss 16.651205
[21,    8] loss 13.709904
[21,    9] loss 14.270817
[21,   10] loss 15.834649
[21,   11] loss 17.457604
[21,   12] loss 17.333244
[21,   13] loss 14.061455
[21,   14] loss 15.251537
[21,   15] loss 14.524360
[21,   16] loss 14.044674
[21,   17] loss 15.970793
[22,    1] loss 15.240684
[22,    2] loss 15.089419
[22,    3] l

[38,    9] loss 14.164148
[38,   10] loss 10.823217
[38,   11] loss 13.884088
[38,   12] loss 16.554945
[38,   13] loss 16.210943
[38,   14] loss 14.451707
[38,   15] loss 14.904278
[38,   16] loss 14.807605
[38,   17] loss 14.967807
[39,    1] loss 14.329214
[39,    2] loss 13.102538
[39,    3] loss 15.135828
[39,    4] loss 14.842800
[39,    5] loss 12.463686
[39,    6] loss 15.648658
[39,    7] loss 14.830049
[39,    8] loss 15.295338
[39,    9] loss 15.364039
[39,   10] loss 14.189307
[39,   11] loss 12.895155
[39,   12] loss 13.939942
[39,   13] loss 17.480331
[39,   14] loss 16.455208
[39,   15] loss 12.516817
[39,   16] loss 12.181778
[39,   17] loss 14.239548
[40,    1] loss 13.607251
[40,    2] loss 14.878580
[40,    3] loss 14.449371
[40,    4] loss 14.111319
[40,    5] loss 13.853257
[40,    6] loss 14.226723
[40,    7] loss 14.115510
[40,    8] loss 12.528407
[40,    9] loss 14.668769
[40,   10] loss 12.779054
[40,   11] loss 15.294491
[40,   12] loss 16.388584
[40,   13] l

In [37]:
##################
#  lr=1e-5
##################
epochnum=0
iteration=0
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(200):
    epochnum+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         ipdb.set_trace()
        iteration+=1
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net_epoch%d-iteration%d.pth'%(epochnum,iteration))

[1,   10] loss 134.299568
[1,   20] loss 132.329617
[1,   30] loss 135.687647
[1,   40] loss 132.201125
[1,   50] loss 136.078483
[1,   60] loss 133.065465
[1,   70] loss 134.326528
[1,   80] loss 132.030489
[1,   90] loss 129.741983
[1,  100] loss 126.645403
[1,  110] loss 129.519817
[1,  120] loss 124.410084
[1,  130] loss 125.953030
[1,  140] loss 128.519710
[1,  150] loss 125.545708
[1,  160] loss 121.291107
[2,   10] loss 119.981345
[2,   20] loss 110.817435
[2,   30] loss 105.803995
[2,   40] loss 98.362232
[2,   50] loss 89.049048
[2,   60] loss 86.946223
[2,   70] loss 74.033331
[2,   80] loss 67.284744
[2,   90] loss 63.127860
[2,  100] loss 48.260866
[2,  110] loss 40.214488
[2,  120] loss 33.705005
[2,  130] loss 28.295592
[2,  140] loss 23.182423
[2,  150] loss 21.661089
[2,  160] loss 19.642079
[3,   10] loss 20.283816
[3,   20] loss 18.566830
[3,   30] loss 18.825332
[3,   40] loss 21.997239
[3,   50] loss 17.086562
[3,   60] loss 18.954543
[3,   70] loss 18.920628
[3,   

[21,   10] loss 15.398592
[21,   20] loss 14.184346
[21,   30] loss 14.559223
[21,   40] loss 14.653625
[21,   50] loss 13.568805
[21,   60] loss 19.832872
[21,   70] loss 18.582728
[21,   80] loss 15.407980
[21,   90] loss 16.185096
[21,  100] loss 15.114405
[21,  110] loss 16.868146
[21,  120] loss 13.496922
[21,  130] loss 13.195046
[21,  140] loss 17.172034
[21,  150] loss 14.979118
[21,  160] loss 15.308647
[22,   10] loss 12.710864
[22,   20] loss 15.011645
[22,   30] loss 15.218285
[22,   40] loss 13.752074
[22,   50] loss 13.418766
[22,   60] loss 15.356521
[22,   70] loss 17.146041
[22,   80] loss 14.713213
[22,   90] loss 15.835199
[22,  100] loss 15.728870
[22,  110] loss 17.430233
[22,  120] loss 13.499685
[22,  130] loss 16.181136
[22,  140] loss 14.150157
[22,  150] loss 14.442425
[22,  160] loss 13.809524
[23,   10] loss 12.966034
[23,   20] loss 13.540684
[23,   30] loss 13.202906
[23,   40] loss 15.274282
[23,   50] loss 13.887502
[23,   60] loss 14.676743
[23,   70] l

[40,  130] loss 14.541113
[40,  140] loss 13.643483
[40,  150] loss 11.010568
[40,  160] loss 11.929895
[41,   10] loss 11.131218
[41,   20] loss 13.791050
[41,   30] loss 13.350237
[41,   40] loss 12.458562
[41,   50] loss 15.574514
[41,   60] loss 12.241973
[41,   70] loss 12.969036
[41,   80] loss 14.381458
[41,   90] loss 14.875616
[41,  100] loss 11.990353
[41,  110] loss 13.243507
[41,  120] loss 11.180364
[41,  130] loss 14.073673
[41,  140] loss 11.537700
[41,  150] loss 14.458933
[41,  160] loss 12.524456
[42,   10] loss 16.146683
[42,   20] loss 11.963037
[42,   30] loss 13.870425
[42,   40] loss 12.125495
[42,   50] loss 12.274877
[42,   60] loss 13.509826
[42,   70] loss 10.899777
[42,   80] loss 13.114007
[42,   90] loss 14.578181
[42,  100] loss 13.751219
[42,  110] loss 12.967936
[42,  120] loss 12.494351
[42,  130] loss 12.262715
[42,  140] loss 13.349425
[42,  150] loss 9.822184
[42,  160] loss 11.622746
[43,   10] loss 13.017532
[43,   20] loss 12.166746
[43,   30] lo

[60,   90] loss 11.583981
[60,  100] loss 12.649878
[60,  110] loss 11.352081
[60,  120] loss 9.260697
[60,  130] loss 14.203349
[60,  140] loss 11.212363
[60,  150] loss 12.598390
[60,  160] loss 10.690963
[61,   10] loss 12.183540
[61,   20] loss 11.997730
[61,   30] loss 11.462947
[61,   40] loss 12.278993
[61,   50] loss 14.276967
[61,   60] loss 13.396092
[61,   70] loss 9.484837
[61,   80] loss 10.540526
[61,   90] loss 13.084164
[61,  100] loss 9.348747
[61,  110] loss 12.296851
[61,  120] loss 13.481287
[61,  130] loss 12.779035
[61,  140] loss 10.793849
[61,  150] loss 10.893150
[61,  160] loss 12.445921
[62,   10] loss 12.816801
[62,   20] loss 13.014676
[62,   30] loss 10.769566
[62,   40] loss 13.093299
[62,   50] loss 12.571456
[62,   60] loss 9.344928
[62,   70] loss 13.896401
[62,   80] loss 10.618738
[62,   90] loss 10.402408
[62,  100] loss 14.865508
[62,  110] loss 12.306596
[62,  120] loss 11.622593
[62,  130] loss 12.241851
[62,  140] loss 10.633454
[62,  150] loss 

[80,   60] loss 9.844217
[80,   70] loss 12.308881
[80,   80] loss 13.002188
[80,   90] loss 11.512635
[80,  100] loss 10.161814
[80,  110] loss 11.579223
[80,  120] loss 13.219178
[80,  130] loss 13.535559
[80,  140] loss 10.344421
[80,  150] loss 9.828506
[80,  160] loss 11.523191
[81,   10] loss 9.905086
[81,   20] loss 9.913093
[81,   30] loss 9.300433
[81,   40] loss 12.022934
[81,   50] loss 9.805184
[81,   60] loss 12.022980
[81,   70] loss 13.615220
[81,   80] loss 9.839054
[81,   90] loss 10.747940
[81,  100] loss 11.701450
[81,  110] loss 10.557802
[81,  120] loss 13.386785
[81,  130] loss 12.899425
[81,  140] loss 9.442037
[81,  150] loss 11.377544
[81,  160] loss 13.024476
[82,   10] loss 11.118600
[82,   20] loss 11.228141
[82,   30] loss 10.180442
[82,   40] loss 10.254639
[82,   50] loss 14.361083
[82,   60] loss 12.181726
[82,   70] loss 11.420161
[82,   80] loss 11.292575
[82,   90] loss 11.563914
[82,  100] loss 11.064414
[82,  110] loss 11.218466
[82,  120] loss 11.2

...

[119,   90] loss 10.576392
[119,  100] loss 11.068437
[119,  110] loss 8.934952
[119,  120] loss 10.753922
[119,  130] loss 9.938462
[119,  140] loss 9.897179
[119,  150] loss 10.553375
[119,  160] loss 9.902129
[120,   10] loss 10.651322
[120,   20] loss 9.216577
[120,   30] loss 10.186345
[120,   40] loss 11.658162
[120,   50] loss 9.317589
[120,   60] loss 8.349532
[120,   70] loss 11.373828
[120,   80] loss 9.794421
[120,   90] loss 8.797920
[120,  100] loss 10.461372
[120,  110] loss 8.749183
[120,  120] loss 8.048030
[120,  130] loss 10.871261
[120,  140] loss 7.777464
[120,  150] loss 11.137135
[120,  160] loss 10.007461
[121,   10] loss 10.277908
[121,   20] loss 10.317077
[121,   30] loss 10.777619
[121,   40] loss 10.208557
[121,   50] loss 9.383306
[121,   60] loss 8.596388
[121,   70] loss 12.327589
[121,   80] loss 9.460010
[121,   90] loss 9.948478
[121,  100] loss 9.248854
[121,  110] loss 10.019310
[121,  120] loss 9.478699
[121,  130] loss 8.913736
[121,  140] loss 9.6

[138,  150] loss 10.002051
[138,  160] loss 8.905482
[139,   10] loss 9.142079
[139,   20] loss 9.222300
[139,   30] loss 10.305054
[139,   40] loss 9.922074
[139,   50] loss 8.545458
[139,   60] loss 8.451187
[139,   70] loss 12.443355
[139,   80] loss 8.938810
[139,   90] loss 8.783118
[139,  100] loss 8.972540
[139,  110] loss 9.072387
[139,  120] loss 8.513771
[139,  130] loss 8.235881
[139,  140] loss 11.129411
[139,  150] loss 8.995359
[139,  160] loss 9.807585
[140,   10] loss 6.728956
[140,   20] loss 8.664526
[140,   30] loss 11.167829
[140,   40] loss 9.645916
[140,   50] loss 10.823834
[140,   60] loss 8.929448
[140,   70] loss 10.641204
[140,   80] loss 9.715105
[140,   90] loss 10.427501
[140,  100] loss 9.220746
[140,  110] loss 10.016709
[140,  120] loss 7.609062
[140,  130] loss 9.013094
[140,  140] loss 9.875535
[140,  150] loss 8.944752
[140,  160] loss 9.996567
[141,   10] loss 8.940572
[141,   20] loss 11.242865
[141,   30] loss 10.315170
[141,   40] loss 8.807459
[

[158,   70] loss 8.324016
[158,   80] loss 9.498942
[158,   90] loss 9.858298
[158,  100] loss 8.591129
[158,  110] loss 8.216933
[158,  120] loss 8.344152
[158,  130] loss 9.273717
[158,  140] loss 7.749646
[158,  150] loss 9.487582
[158,  160] loss 9.168487
[159,   10] loss 9.371869
[159,   20] loss 9.346002
[159,   30] loss 8.764380
[159,   40] loss 8.278126
[159,   50] loss 9.602832
[159,   60] loss 7.823578
[159,   70] loss 9.522020
[159,   80] loss 9.240379
[159,   90] loss 8.719040
[159,  100] loss 8.796643
[159,  110] loss 8.908282
[159,  120] loss 9.235514
[159,  130] loss 10.563823
[159,  140] loss 9.454334
[159,  150] loss 10.171915
[159,  160] loss 9.827206
[160,   10] loss 9.011468
[160,   20] loss 9.554752
[160,   30] loss 7.529510
[160,   40] loss 10.924004
[160,   50] loss 7.946727
[160,   60] loss 9.690063
[160,   70] loss 10.564862
[160,   80] loss 8.953710
[160,   90] loss 10.347947
[160,  100] loss 9.149051
[160,  110] loss 8.080402
[160,  120] loss 7.286139
[160,  

[178,   10] loss 9.436863
[178,   20] loss 8.266971
[178,   30] loss 7.635916
[178,   40] loss 8.726009
[178,   50] loss 8.551650
[178,   60] loss 7.901505
[178,   70] loss 9.719926
[178,   80] loss 8.977957
[178,   90] loss 8.480833
[178,  100] loss 7.354341
[178,  110] loss 9.357048
[178,  120] loss 8.884423
[178,  130] loss 8.904060
[178,  140] loss 8.305448
[178,  150] loss 9.614554
[178,  160] loss 7.112962
[179,   10] loss 9.669504
[179,   20] loss 8.622600
[179,   30] loss 9.760108
[179,   40] loss 7.933162
[179,   50] loss 11.059308
[179,   60] loss 8.607331
[179,   70] loss 7.192022
[179,   80] loss 9.297703
[179,   90] loss 9.746862
[179,  100] loss 8.912885
[179,  110] loss 7.972771
[179,  120] loss 7.740001
[179,  130] loss 6.271955
[179,  140] loss 7.975037
[179,  150] loss 7.406565
[179,  160] loss 8.494843
[180,   10] loss 9.074404
[180,   20] loss 9.885044
[180,   30] loss 7.945162
[180,   40] loss 8.282041
[180,   50] loss 7.226470
[180,   60] loss 10.012128
[180,   70

[197,  120] loss 6.582457
[197,  130] loss 7.855511
[197,  140] loss 8.905802
[197,  150] loss 8.791404
[197,  160] loss 10.044801
[198,   10] loss 8.313948
[198,   20] loss 9.601603
[198,   30] loss 7.645288
[198,   40] loss 9.277590
[198,   50] loss 8.305311
[198,   60] loss 7.034502
[198,   70] loss 6.582784
[198,   80] loss 7.428387
[198,   90] loss 9.714935
[198,  100] loss 8.899665
[198,  110] loss 8.294057
[198,  120] loss 7.265180
[198,  130] loss 8.886294
[198,  140] loss 7.673786
[198,  150] loss 7.509605
[198,  160] loss 9.794825
[199,   10] loss 8.029254
[199,   20] loss 8.401737
[199,   30] loss 8.079035
[199,   40] loss 9.560243
[199,   50] loss 8.840394
[199,   60] loss 7.428243
[199,   70] loss 6.892544
[199,   80] loss 7.415276
[199,   90] loss 8.904389
[199,  100] loss 9.062617
[199,  110] loss 6.973261
[199,  120] loss 6.220614
[199,  130] loss 9.227940
[199,  140] loss 8.192176
[199,  150] loss 7.423880
[199,  160] loss 8.726039
[200,   10] loss 8.860398
[200,   20]

In [10]:
#####################
# 再训练20epoch 加大batcsize
#####################
optimizer=optim.Adam(model.parameters(),lr=1e-7)
Net=model.double()
Net.load_state_dict(t.load('/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net_epoch200-iteration32600.pth'))
batch_size=100
epochnum=100
iteration=32600
dloador=DataLoader(dataloader,batch_size=batch_size,shuffle=True,num_workers=8,pin_memory=CUDA)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(20):
    epochnum+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
#         ipdb.set_trace()
        iteration+=1
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#         if i%10==9:
        print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss))
        running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net_epoch%d-iteration%d_batchsize%d.pth'%(epochnum,iteration,batch_size))

[1,    1] loss 18.075403
[1,    2] loss 19.515793
[1,    3] loss 17.558434
[1,    4] loss 16.412480
[1,    5] loss 18.079704
[1,    6] loss 19.425593
[1,    7] loss 20.072812
[1,    8] loss 18.760013
[1,    9] loss 18.147868
[1,   10] loss 18.618706
[1,   11] loss 17.294487
[1,   12] loss 19.384800
[1,   13] loss 17.303515
[1,   14] loss 18.040458
[1,   15] loss 18.887307
[1,   16] loss 16.524797
[1,   17] loss 13.260293
[2,    1] loss 17.287551
[2,    2] loss 17.788845
[2,    3] loss 18.117845
[2,    4] loss 20.128804
[2,    5] loss 17.015121
[2,    6] loss 17.332191
[2,    7] loss 17.894420
[2,    8] loss 14.809302
[2,    9] loss 18.031544
[2,   10] loss 18.067026
[2,   11] loss 17.439267
[2,   12] loss 19.546238
[2,   13] loss 17.064752
[2,   14] loss 18.226314
[2,   15] loss 18.054190
[2,   16] loss 18.959108
[2,   17] loss 11.341774
[3,    1] loss 19.969725
[3,    2] loss 17.759180
[3,    3] loss 17.005852
[3,    4] loss 18.858227
[3,    5] loss 18.909532
[3,    6] loss 16.703894


KeyboardInterrupt: 

In [40]:
###############################
# 模型实时测试 在12组数据(训练)上
###############################
testfilepath='/home/wcj/ReferenceProject/PPGnet/ppghr'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=model(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1622个样本计算MAE: 25.352355


In [41]:
###############################
# 模型实时测试 在10组数据上
###############################

testfilepath='/home/wcj/ReferenceProject/PPGnet/testdata'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=model(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1328个样本计算MAE: 28.195536


In [40]:
########################
# leave one
########################

Net=PPGNet(kernels,125,125,125).eval()
Net.double()
Net.load_state_dict(t.load('/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net_epoch200-iteration32600.pth'))

testfilepath='/home/wcj/ReferenceProject/PPGnet/12thData'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 146个样本计算MAE: 24.093812


In [41]:
Net=PPGNet(kernels,125,125,125).eval()
Net.double()
Net.load_state_dict(t.load('/home/wcj/ReferenceProject/PPGnet/checkpoints/0429Net_epoch20_lr1e-05_weight_decay.pth'))

testfilepath='/home/wcj/ReferenceProject/PPGnet/testdata'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1328个样本计算MAE: 38.290428


In [43]:
###############################
# 模型测试 在12组数据(训练)上
###############################
Net=PPGNet(kernels,125,125,125).eval()
Net.double()
Net.load_state_dict(t.load('/home/wcj/ReferenceProject/PPGnet/checkpoints/0429Net_epoch100_lr1e-05_weight_decay_1.pth'))
testfilepath='/home/wcj/ReferenceProject/PPGnet/ppghr'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=Net(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1622个样本计算MAE: 17.072523
